*For any questions email: santanavidda@gmail.com*

# Location of current data

## Binetflow files

Current binetflow files are located in (under d_santan home) `~/ddos-detection/binetflows` They are the raw binetflow files from the CTU-13 dataset and are numbered
as such. Use these files to aggregate the data.

## Aggregated data

The currently generated data is located in (under d_santan home) `~/ddos-detection/minute_aggregated`. The naming scheme of the files in there is
`[attack_type]-[interval]s.featureset.csv`. The files with background connections are the ones with `_background` after`[attack_type]` in the name.

There is also some files in `/6TB1/david/files/`. Currently, there are background files for each attack type for 0.01 second intervals, which is the recommended
interval for classifying background connections.

# Generating feature set

The code used to aggregate the binetflow files is in `main.py`. There are built in command line parameters that will allow you to dictate
the attack type, interval, and other options for aggregation. Some examples:

```bash
" Will aggregate all DDOS files on one second intervals
python3 main.py --attack_type=ddos --interval=1

" Will aggregate all SPAM files on 0.01 second interval and includes background connections
python3 main.py --attack_type=spam --interval=0.01 --use_background
```

The results are stored in the folder `minute_aggregated/` under
the format `{attack name}-{interval}s.featureset.csv`. This is also where all previously aggregated data is located.

**NOTE:** By default, `main.py` excludes background connections. To include background connections, use the flag `--use_background`.

All flags for `main.py`

```bash
  --attack_type: Type of files to aggregate together.
  --custom_suffix: Just for debug
    (default: '')
  --interval: Interval in seconds to aggregate connections.
    (a number)
  --[no]norm_and_standardize: To normalize and standardize the feature values.
    (default: 'false')
  --[no]single: Whether this is aggregating a single file or not.
    (default: 'false')
  --[no]use_background: To include background connections to the aggregation.
    (default: 'false')
  --[no]use_separator: Whether this is aggregating a single file or not.
    (default: 'false')
```

## Features used 
The features that are used can be found in the `summarizer.py` script. The full list of features that are written to the csv file are under the class `Summarizer` on the field `features`. You can see the features we used below

In [2]:
from summarizer import Summarizer
print(Summarizer().features)

['n_conn', 'avg_duration', 'n_udp', 'n_tcp', 'n_icmp', 'n_sports>1024', 'n_sports<1024', 'n_dports>1024', 'n_dports<1024', 'n_d_a_p_address', 'n_d_b_p_address', 'n_d_c_p_address', 'n_d_na_p_address', 'std_packets', 'std_bytes', 'std_time', 'std_srcbytes', 'src_to_dst', 'entropy_sports>1024', 'entropy_sports<1024', 'entropy_dports>1024', 'entropy_dports<1024', 'entropy_srcport', 'entropy_dstport', 'entropy_dstip', 'n_s_a_p_address', 'n_s_b_p_address', 'n_s_c_p_address', 'n_s_na_p_address', 'entropy_srcip', 'entropy_src_a_ip', 'entropy_src_b_ip', 'entropy_src_c_ip', 'entropy_src_na_ip', 'entropy_dst_a_ip', 'entropy_dst_b_ip', 'entropy_dst_c_ip', 'entropy_dst_na_ip', 'entropy_bytes', 'entropy_src_bytes', 'entropy_time', 'entropy_state', 'entropy_packets']


Removing or adding to the that array will effect what is written to the output csv.

The summarizer class has an `add()` function which takes a dictionary object that is basically a line from a botnet file, and it will add it to the overall features
to it's appropriate window. For example, the function will look at the value `dur` and add it and average the result to the `avg_duration` feaure found in the features array. 
So if there is another feature you want to implement, look at the `add()` function where you'll get a line from botnet file and use the information provided appropriately.

# Training the models

There is a script, `botnet_detection_main.py`, that has some functions set up to train some models. You can train Random Forest and Deep Learning models
using their default parameters using the command line arguments of the script. Here's an example:

```bash
" Will run a random forest model on the file /minute_aggregated/ddos-1.0s.featureset.csv
python3 botnet_detection_main.py --attack_type=ddos --interval=1 --model_type=rf

" Will run a Deep Learning model on the file /minute_aggregated/spam_background-0.01s.featureset.csv
python3 botnet_detection_main.py --attack_type=spam --interval=0.01 --use_background --model_type=dl
```

All models are trained using a 70/30 
train/test split. You can use `--model_type=dl` to use Deep Learning instead. The results will be printed which will contain the accuracy, precision, recall, and F1 
score

**NOTE:** The command line arguemnts here are similar to `main.py`. So by default, the script will look for files that excluded background connections. To make it look for files
with background connections, user the flag `--use_background`.

If you want to change the parameters of the models, you will have to go into the code in `botnet_detection.py`. There is two functions you'll want to look at: `rf_train()` and `dl_train()`. 
These functions create the models for their respective models so you can look into them to change parameters to how you wan them.


### All flags for botnet_detection_main.py:

```bash
  --attack_type: Type of attack to train on.
  --interval: Interval of the file to train on.
    (a number)
  --model_type: Type of model to train with.
  --[no]norm_and_standardize: To normalize and standardize the features
    (default: 'false')
  --[no]sample: Whether or not to sample from Normal labels.
    (default: 'false')
  --[no]use_ahead: Whether or not to use attack as the label
    (default: 'false')
  --[no]use_attacks: Whether or not to use attack as the label
    (default: 'false')
  --[no]use_background: Use the file that has background information.
    (default: 'false')
  --[no]use_bots: Whether or not to use bots as the label.
    (default: 'false')

```
